SLiMe is a package that allows the user to simulate large-scale genomic data under realistically complex demographic histories while retaining information about local ancestry.

This notebook describes how to use SLiMe.

### Basic procedure

1. Recent history since the start of admixture is simulated forward-in-time with `SLiM`.
2. Older history of the ancestral populations is simulated backwards-in-time with `msprime`.

## Necessary inputs

**Recombination map**

A recombination map for the admixed history can be given. 
The file should look like the one below.
Ensure that the final row of the recombination map corresponds to the length of the chromosome that you wish to simulate.

```
Chromosome position COMBINED_rate(cM/Mb) Genetic_Map(cM)
22 16051347 9.6640973708 0
22 16052618 9.7078062447 0.0123386217370137
22 16053624 9.7138922111 0.0221107973013803
22 16053659 9.716343506 0.0224508693240903
22 16053758 9.707808785 0.0234119423938053
22 16054713 9.5801157234 0.0325609529096523
22 16054960 9.5805112114 0.0349273391788681
.
.
.
22 49991555 0.1286194771 73.1460580463483
22 49991660 0.1286937712 73.1460715591943
22 49992695 0.1297177603 73.1462058170762
22 49993745 0.1310658564 73.1463434362254
22 49994152 0.1310658564 73.146396780029
22 49997158 0.1310658564 73.1467907639933
22 49997614 0.1310667948 73.1468505304517
22 49997706 0.1313729993 73.1468626167677
22 49997967 0.1313729993 73.1468969051205
22 50000000 0.1315909877 73.1470166529193
```

### Packages

In [1]:
import slime, msprime, pyslim

(Note: For now I've installed my local version of tskit that allows unary nodes to kept while simplifying - but use the proper version once the next tskit release is out!!!)

In [2]:
import sys
!{sys.executable} -m pip install tskit-0.1.6.dev0.tar.gz

Processing ./tskit-0.1.6.dev0.tar.gz
  Running setup.py bdist_wheel for tskit ... done
  Stored in directory: /Users/gtsambos/Library/Caches/pip/wheels/ee/67/b5/0a568d34bc8bb222f35ac2c0dd4e3152f678f1595763d27df1
Successfully built tskit
  Found existing installation: tskit 0.1.6.dev0
    Uninstalling tskit-0.1.6.dev0:
      Successfully uninstalled tskit-0.1.6.dev0


In [3]:
import tskit

In [4]:
# !{sys.executable} -m pip install msprime --upgrade

## Step 1: Simulate recent history forward-in-time with SLiM.

See the [SLiM Manual](http://benhaller.com/slim/SLiM_Manual.pdf) for how to use the Eidos scripting language.

Here's a script for a simple two-way admixture under neutrality.

In [ ]:
slim_script = '''
// set up a simple neutral simulation
initialize() {

//	setwd("/Users/gtsambos/Projects/AdmixtureSim/5.msprime.and.slim");
	
	// Set constants.
	defineConstant("chromLength", 49999999);
	defineConstant("p0Size", 1000);
	defineConstant("p1Size", 1000);
	defineConstant("p2Size", 1000);
	defineConstant("p0Prop", 10/100);
	defineConstant("p1Prop", 1 - p0Prop);

	initializeSLiMModelType("WF");
	initializeSLiMOptions(keepPedigrees = T); // Keep pedigree information
	initializeTreeSeq();
	initializeMutationRate(0);
	
	// m1 mutation type: neutral
	initializeMutationType("m1", 0.5, "f", 0.0);
	
	// g1 genomic element type: uses m1 for all mutations
	initializeGenomicElementType("g1", m1, 1.0);
	
	// uniform chromosome of length 100 kb
	initializeGenomicElement(g1, 0, chromLength);
	
	// read in chr22 recombination map.
	// See example 6.1.2 in the SLiM manual.
	lines = readFile("examples/chr22.recombination_map");
	lines = lines[1:(size(lines)-1)]; // remove header
	rates = NULL;
	ends = NULL;
	
	for (line in lines)
	{
		components = strsplit(line, " ");
		ends = c(ends, asInteger(components[1]));
		rates = c(rates, asFloat(components[2]));
	}
	
	ends = ends - 1; // final coordinate in file is 50 000 000
	rates = rates * 1e-8;
	
	initializeRecombinationRate(rates, ends);
	
}

1 early(){
	sim.addSubpop("p0", p0Size);
	sim.addSubpop("p1", p1Size);

//	sim.treeSeqRememberIndividuals(sim.subpopulations.individuals);
}

1 late() { 
	
	// Add admixing population.
	sim.addSubpop("p2", p2Size);
	p2.setMigrationRates(c(p0,p1),c(p0Prop, p1Prop));
	
}


2 late() {

	p2.setMigrationRates(c(p0,p1),c(0.0,0.0));

}


// save the tree output after 200 generations.
200 late() {
 	sim.treeSeqOutput("examples/recent-history.trees");
	sim.simulationFinished();
}
'''

slim_file = open("examples/slim_script.slim", "w")
slim_file.writelines(slim_script)
slim_file.close()

The `simulate_recent_history` function is a wrapper for running SLiM via python.

In [ ]:
slime.simulate_recent_history("examples/slim_script.slim", outFile = "examples/recent-history.trees",
                             logFile = "examples/recent-history.log")

Cool. Time to move on.

## Step 2 (optional): subsample population

`SLiM` generates a tree sequence that holds the chromosomes of *entire populations*. If you are interested in only a subset of the simulated population, this tree sequence will contain a lot of redundant history. In this step, I choose a random sample of individuals from the generated tree sequence and simplify the tree sequence so that only the history relevant to these samples is retained.

Note: the below *needs* to be `tskit`, because the correct version of simplify isn't available for `msprime` tree sequences.

In [ ]:
ts = tskit.load("examples/recent-history.trees")

In [ ]:
new_ts = slime.TreeSequenceToSample(
    ts = ts,
    populations_to_sample_from = [2],
    sample_sizes = [10]
)

In [ ]:
ts = new_ts.subsample()

In [ ]:
ts.dump("examples/script-2.trees")

## Step 3: recapitation

Next, we add the ancient history of the samples. This is generated with `msprime`.
This step is basically just a wrapper for the `recapitate` function in `pyslim`.

To do this, we need to add in some parameters pertaining to the ancient history of our sample.

Note that, atm, you must specify a uniform recombination rate for the ancient history. This [may change in the future](https://github.com/tskit-dev/pyslim/blob/master/README.md#things-you-cannot-do).

Another error that I should put in here is population sizes - it would be very easy to mess this up, especially someone who doesn't reallly understand that one of the methods does real and the other does effective population sizes.

Can you automate the creation of the population configurations to get over this problem.

** This is something I should consider doing! Do it later **

In [ ]:
rho = 1e-8 # recombination rate. 
N0 = 1e4 # population size
divergenceGen = 1000

In [ ]:
######################
# POPULATION HISTORY #
######################


# Population IDs
population_configurations = [
    # CEU
    msprime.PopulationConfiguration(
      #  sample_size = sampleSize, 
        initial_size = N0,
        growth_rate = 0),
    # YRI
    msprime.PopulationConfiguration(
       # sample_size = sampleSize,                                
        initial_size = N0,                                
        growth_rate = 0),
    # ASW - needed as a 'dummy'
    msprime.PopulationConfiguration(
        sample_size = 0,                                                              
        growth_rate = 0)
]

demographic_events = [
	msprime.MassMigration(
		time=divergenceGen,
		source = 1,
		destination = 0,
		proportion = 1.0),
	msprime.MigrationRateChange(time = divergenceGen,
		rate = 0)
]

Weird. what is different about loading in the tree seq, versus just carrying it on from the previous operation?

The difference seems to be that one keeps the treeSeq as a `tskit` object, whereas loading the file in uses the msprime library and so ensures that a `msprime`-compliant tree is used.

Ah - no - it was my version of msprime. All fixed now!

Ideally, everything that you can do under ecapitate can be done here as well.

In [ ]:
ts1 = pyslim.SlimTreeSequence.load_tables(ts.tables)


In [ ]:
recap = ts1.recapitate(
	recombination_rate=rho,
	population_configurations = population_configurations,
	demographic_events=demographic_events,
    keep_first_generation = True
	)

As written (ie. without extra checks), this is currently probably runnable without having any extra wrapper functions. (Recapitate is already a wrapper function of sorts). 

## Step 4: add in variation

For the moment, let's assume all variation is neutral.

In [ ]:
mu = 1e-8 # rate of neutral mutations

In [ ]:
mut_ts = pyslim.SlimTreeSequence(msprime.mutate(recap, rate=mu, keep=True))

## Wrapper for the simulation

In [ ]:
import slime

In [ ]:
mysim = slime.AdmixtureSimulation("examples/slim_script.slim", 
                                  populations_to_sample_from = [2], sample_sizes = [10])

In [ ]:
ts = mysim.debugger()

In [ ]:
ts1 = mysim.go()